# Simple Statistical Tool App using Shiny

This is a simple simple statistical tool that has the following functions as of now:
* upload a data set of different format.(works great with csv files)
* calculate the measure of location such as mean, mode, median, standard divaiation and others.
* plot chart by entering column name (some chart are bar, scatter, histogram)

There are other two features that I am working on to add. These are:
* to make it easy for the user to create its own data set, by specifying number of columns and rows it needs
* to be able to connect any data set to google spreed sheet or excel for editing

all contribution are greatly appreaciated.



### required libraries are below

In [3]:
# install.packages("shiny", repos = "https://cloud.r-project.org/")
# install.packages("tidyverse", repos = "https://cloud.r-project.org/")


package 'shiny' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\isaac\AppData\Local\Temp\RtmpWqSPtr\downloaded_packages
package 'tidyverse' successfully unpacked and MD5 sums checked

The downloaded binary packages are in
	C:\Users\isaac\AppData\Local\Temp\RtmpWqSPtr\downloaded_packages


#### The UI code ( will saved in R studio as ui.R)

In [3]:
#ui.R file


#import shiny
library(shiny)

#create the UI of the app ( the view or template)

shinyUI(fluidPage(
  titlePanel("Simple Statistics Tool App"),
  
  #sidebarPanel create a box to group other elemets in it
  sidebarPanel(
    fileInput('fileId', 'Choose file to upload',accept = c('text/csv','text/comma-separated-values','text/tab-separated-values','text/plain','.csv','.tsv')),
    checkboxInput('headerId', 'Header', TRUE),
    radioButtons('sepId', 'Separator',c(Comma=',',Semicolon=';',Tab='\t'),','),
    radioButtons('quoteId', 'Quote',c(None='','Double Quote'='"','Single Quote'="'"),selected = '"'),
    radioButtons('showId', 'Display Data',c('Show Less' = 1,'Show more' = 2)),
    actionButton("LoadId", "Load File"),width = 3),
  
  sidebarPanel(
    textInput("colNameId",label = "Enter column name to calculate",value = ""),
    radioButtons('measureId', 'Measure of Location and Dispersion',c(Mean = 1, Mode = 2, Median = 3, Varient = 4, "Standard Devaition" = 5, Summary = 6)),
    verbatimTextOutput("measure_result")
  ),
  
  sidebarPanel(
    textInput("xId",label = "Enter column name from data set for x",value = ""),
    textInput("yId",label = "Enter column name from data set for y",value = ""),
    radioButtons('graphId', 'Charts',c(Bar = 1, "Scatter (enter x and y value)" = 2, Histogram = 3, "Boxplot (enter x and y value)" = 4)),
    actionButton("plotIt","Plot")
  ),
    
  mainPanel(
    tabsetPanel(
      tabPanel("Imported Data", tableOutput("my_output_data")),
      tabPanel("Graph", plotOutput("charts_result")),
      tabPanel("Add Data")
      )
  )
  
))


<div class="container-fluid">
  <h2>Simple Statistics Tool App</h2>
  <div class="col-sm-3">
    <form class="well">
      <div class="form-group shiny-input-container">
        <label>Choose file to upload</label>
        <div class="input-group">
          <label class="input-group-btn">
            <span class="btn btn-default btn-file">
              Browse...
              <input id="fileId" name="fileId" type="file" style="display: none;" accept="text/csv,text/comma-separated-values,text/tab-separated-values,text/plain,.csv,.tsv"/>
            </span>
          </label>
          <input type="text" class="form-control" placeholder="No file selected" readonly="readonly"/>
        </div>
        <div id="fileId_progress" class="progress progress-striped active shiny-file-input-progress">
          <div class="progress-bar"></div>
        </div>
      </div>
      <div class="form-group shiny-input-container">
        <div class="checkbox">
          <label>
            <input id="h

#### The server code (will saved in Rstudio as server.R)

In [6]:
# load libries
library(shiny)
library(tidyverse)

# Define server logic
shinyServer(function(input, output){
  
  # mode function( most frequent element)
  Mode <- function(x) {
    x <- as.numeric(unlist(x))
    x <- lapply(x, Filter, f = negate(is.na))
    ux <- unique(x)
    ux[which.max(tabulate(match(x, ux)))]
  }
  
  data <- reactive({
    if(input$LoadId == 0){return("No file has been uploaded. Please upload a file")}
    inFile <- input$fileId
    if (is.null(inFile)){return(NULL)}
    
    isolate({ 
      input$LoadId
      my_data <- read.csv(inFile$datapath, header = input$headerId,sep = input$sepId, quote = input$quoteId,stringsAsFactors = FALSE)
    })
    
    if(input$colNameId == "" ){
      outdata <- my_data
      outdata
    }else{
      outdata <- my_data[input$colNameId]
      outdata
    }
    
    if(input$showId == 1){
      head(outdata, 15)
    }else{
      outdata
    }
    
  })
  
  #function for calculating he measure of location
  measure <- reactive({
    if(input$measureId == 1 & input$colNameId != ""){
      sapply(data(), mean, na.rm = TRUE)
      
    } else if(input$measureId == 2 & input$colNameId != ""){
      sapply(data(), Mode)
      
    }else if(input$measureId == 3 & input$colNameId != ""){
      sapply(data(),median,na.rm = TRUE)
      
    }else if(input$measureId == 4 & input$colNameId != ""){
      sapply(data(),var,na.rm = TRUE)
      
    }else if(input$measureId == 5 & input$colNameId != ""){
      sapply(data(),sd, na.rm = TRUE)
      
    }else if(input$measureId == 6 & input$colNameId != ""){
      sapply(data(),summary)
    }
  })
  
  chart <- eventReactive(input$plotIt,{
    if(input$LoadId == 0){return("No file has been uploaded. Please upload a file")}
    inFile <- input$fileId
    if (is.null(inFile)){return(NULL)}
    
    if(input$xId != "" | input$yId != ""){
      isolate({ 
        input$LoadId
        my_data <- read.csv(inFile$datapath, header = input$headerId,sep = input$sepId, quote = input$quoteId,stringsAsFactors = FALSE)
      })
    }
    
    #convert x and y values to double from list after file check and load from above
    xval <- as.numeric(unlist(my_data[input$xId]))
    yval <- as.numeric(unlist(my_data[input$yId]))
    
    #check which radio is clicked and plot its apporpiate graph
    if(input$graphId == 1){
      
      ggplot(my_data) + geom_bar(mapping = aes(x = xval, fill = I("blue")))
      
    }else if(input$graphId == 2){
      ggplot(my_data) + geom_point(mapping = aes(x = xval, y = yval), color = "blue")
      
    }else if(input$graphId == 3){
      ggplot(my_data,aes(x = xval,fill = I("blue"))) + geom_histogram(binwidth = 0.5)
      
    }else if(input$graphId == 4){
      ggplot(my_data,mapping = aes(x = xval,y = yval)) + 
        geom_boxplot(aes(group = cut_width(xval, 0.25)),fill = "white", colour = "#3366FF")
    }
  })
  
  output$my_output_data <- renderTable({data()},include.rownames=FALSE) 
  output$measure_result <- renderPrint({measure()})
  output$charts_result <- renderPlot({chart()})
  
})

#### Thank you!
